In [3]:
from birdset.datamodule import DatasetConfig
from birdset.datamodule.birdset_datamodule import BirdSetDataModule
from birdset.datamodule import LoadersConfig, LoaderConfig

dm = BirdSetDataModule(
    dataset=DatasetConfig(
        data_dir="./datasets",
        hf_path="DBD-research-group/BirdSet",
        hf_name="POW",
        n_workers=21,
        val_split=0.2,
        task="multiclass",
        classlimit=500,
        eventlimit=5,
        sample_rate=32000,
    ),
    loaders=LoadersConfig(
        train=LoaderConfig(batch_size=8, shuffle=True),
        valid=LoaderConfig(batch_size=8, shuffle=False),
        test=LoaderConfig(batch_size=8, shuffle=True),
    ),
)

dm.prepare_data()
dm.setup(stage="fit")

train_loader = dm.train_dataset
validation_loader = dm.val_dataset


c:\Users\johnd\anaconda3\envs\birdset\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\johnd\anaconda3\envs\birdset\lib\site-packages\torch_audiomentations\utils\io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
sampling: 100%|██████████| 48/48 [00:04<00:00, 10.59it/s]


Saving the dataset (0/1 shards):   0%|          | 0/41115 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10279 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16052 [00:00<?, ? examples/s]

In [10]:
import tensorflow as tf
import numpy as np

num_classes = dm.num_classes

def data_generator(dataset, augment=False):
    for sample in dataset:
        # Get audio and label
        audio = sample['input_values']
        label = sample['labels']
        
        # Handle the shape - BirdSet returns shape (1, 32000) or (batch, 32000)
        if isinstance(audio, np.ndarray):
            audio = audio.flatten()  # Force to 1D
        else:
            audio = audio.numpy().flatten()  # Force to 1D
        
        # Ensure audio is exactly 1D with 32000 samples
        if audio.shape[0] != 32000:
            # Pad or trim to 32000
            if audio.shape[0] < 32000:
                audio = np.pad(audio, (0, 32000 - audio.shape[0]), mode='constant')
            else:
                audio = audio[:32000]
        
        # Extract label and convert to int
        if isinstance(label, np.ndarray):
            label = int(label.item() if label.size == 1 else label.flat[0])
        else:
            label = int(label.numpy().item() if label.numpy().size == 1 else label.numpy().flat[0])
        
        audio_tensor = tf.convert_to_tensor(audio, dtype=tf.float32)
        audio_tensor = tf.reshape(audio_tensor, [-1])
        
        # Compute STFT
        stft = tf.signal.stft(audio_tensor, frame_length=2048, frame_step=512, fft_length=2048)
        spectrogram = tf.abs(stft)
        
        # Convert to mel scale (80 mel bins is standard)
        num_spectrogram_bins = spectrogram.shape[-1]
        lower_edge_hertz, upper_edge_hertz = 80.0, 7600.0
        num_mel_bins = 128
        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins, num_spectrogram_bins, 32000, lower_edge_hertz, upper_edge_hertz
        )
        mel_spectrogram = tf.tensordot(spectrogram, linear_to_mel_weight_matrix, 1)
        mel_spectrogram.set_shape(spectrogram.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))
        
        # Convert to log scale (dB)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)
        
        # Convert to numpy
        spectrogram = log_mel_spectrogram.numpy()
        
        # Transpose to (time, mel_bins)
        spectrogram = spectrogram.T
        
        # Add channel dimension
        spectrogram = spectrogram[..., np.newaxis]
        
        # Resize to 224x224x1
        spectrogram_resized = tf.image.resize(spectrogram, [224, 224], method='bilinear').numpy()
        
        # Repeat to 3 channels
        spectrogram_3ch = np.repeat(spectrogram_resized, 3, axis=-1)

        # Normalize to [0, 1]
        spec_min = spectrogram_3ch.min()
        spec_max = spectrogram_3ch.max()
        spectrogram_3ch = (spectrogram_3ch - spec_min) / (spec_max - spec_min + 1e-8)

        # One-hot encode labels
        label_onehot = tf.one_hot(label, num_classes).numpy()

        if augment:
            # Random time/frequency masking (SpecAugment)
            spectrogram_3ch = tf.image.random_brightness(spectrogram_3ch, 0.2).numpy()
            
            # Random horizontal flip (time reversal)
            if np.random.random() > 0.5:
                spectrogram_3ch = np.flip(spectrogram_3ch, axis=1)
        
        yield spectrogram_3ch.astype(np.float32), label_onehot.astype(np.float32)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(dm.train_dataset, augment=False),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32),
    )
).batch(8).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(dm.val_dataset),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(num_classes,), dtype=tf.float32),
    )
).batch(8).prefetch(tf.data.AUTOTUNE)

In [12]:
# We want to use Tensorflow Keras for the model
import keras as ks
from keras.layers import Dense, Dropout

num_classes = dm.num_classes

# Copy efficientnetb0 architecture
model = ks.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)

x = model.output
x = Dropout(0.3)(x)
x = Dense(num_classes, activation='softmax')(x)
model = ks.models.Model(inputs=model.input, outputs=x)


In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
                metrics=['accuracy'])

# let's train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=callbacks)


Epoch 1/10


C:\Users\johnd\BirdSet\birdset\datamodule\components\transforms.py:172: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float16)


5140/5140 [==============================] - 3736s 726ms/step - loss: 3.5826 - accuracy: 0.1110 - val_loss: 6.1050 - val_accuracy: 0.0979 - lr: 0.0010
Epoch 2/10
5140/5140 [==============================] - 3658s 712ms/step - loss: 3.4590 - accuracy: 0.1230 - val_loss: 3.6329 - val_accuracy: 0.1166 - lr: 0.0010
Epoch 3/10
5140/5140 [==============================] - 3709s 722ms/step - loss: 3.3831 - accuracy: 0.1408 - val_loss: 3.3785 - val_accuracy: 0.1437 - lr: 0.0010
Epoch 4/10
5140/5140 [==============================] - 3682s 716ms/step - loss: 3.3108 - accuracy: 0.1526 - val_loss: 3.3914 - val_accuracy: 0.1390 - lr: 0.0010
Epoch 5/10
5140/5140 [==============================] - 3716s 723ms/step - loss: 3.2548 - accuracy: 0.1644 - val_loss: 3.3609 - val_accuracy: 0.1518 - lr: 0.0010
Epoch 6/10
5140/5140 [==============================] - 3705s 721ms/step - loss: 3.1968 - accuracy: 0.1759 - val_loss: 3.2272 - val_accuracy: 0.1776 - lr: 0.0010
Epoch 7/10
5140/5140 [=================

In [15]:
model.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=callbacks)

Epoch 1/10
5140/5140 [==============================] - 3714s 722ms/step - loss: 1.9825 - accuracy: 0.4587 - val_loss: 3.1959 - val_accuracy: 0.2586 - lr: 5.0000e-04
Epoch 2/10
5140/5140 [==============================] - 3835s 746ms/step - loss: 1.8749 - accuracy: 0.4886 - val_loss: 3.3088 - val_accuracy: 0.2563 - lr: 5.0000e-04
Epoch 3/10
5140/5140 [==============================] - 3988s 776ms/step - loss: 1.7830 - accuracy: 0.5128 - val_loss: 3.2372 - val_accuracy: 0.2445 - lr: 5.0000e-04
Epoch 4/10
5140/5140 [==============================] - 3804s 740ms/step - loss: 1.6789 - accuracy: 0.5381 - val_loss: 3.9953 - val_accuracy: 0.2093 - lr: 5.0000e-04
Epoch 5/10
5140/5140 [==============================] - 3841s 747ms/step - loss: 1.4983 - accuracy: 0.5912 - val_loss: 3.2759 - val_accuracy: 0.2731 - lr: 2.5000e-04
Epoch 6/10
5140/5140 [==============================] - 4042s 786ms/step - loss: 1.3960 - accuracy: 0.6193 - val_loss: 3.1806 - val_accuracy: 0.2955 - lr: 2.5000e-04
Epoc